In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
import IPython.display as ipd
import plotly.express as px
import librosa.display
import pandas as pd
import numpy as np
import librosa
import warnings
import IPython
import os
from tqdm import tqdm

plt.style.use("ggplot")
warnings.filterwarnings(action='ignore')

In [ ]:
train = pd.read_csv("../input/birdsong-recognition/train.csv")
train.info()

In [ ]:
def get_top_data_info(df, sort_items, top_rate, min_sample_num = 10):
    for k,v in sort_items.items():
        if v == "up":
            df = df.sort_values(k)
        else:
            df = df.sort_values(k, ascending=False)
    # get top data
    length = len(df)
    sample_num = int(length * top_rate)
    if sample_num < min_sample_num:
        top_df = df.iloc[:]
    else:
        top_df = df.iloc[:sample_num]
    return top_df

import numpy as np
from scipy import signal
 
#バターワースフィルタ（ハイパス）
def highpass(x, samplerate, fp, fs, gpass, gstop):
    fn = samplerate / 2                           #ナイキスト周波数
    wp = fp / fn                                  #ナイキスト周波数で通過域端周波数を正規化
    ws = fs / fn                                  #ナイキスト周波数で阻止域端周波数を正規化
    N, Wn = signal.buttord(wp, ws, gpass, gstop)  #オーダーとバターワースの正規化周波数を計算
    b, a = signal.butter(N, Wn, "high")           #フィルタ伝達関数の分子と分母を計算
    y = signal.filtfilt(b, a, x)                  #信号に対してフィルタをかける
    return y                                      #フィルタ後の信号を返す

#バターワースフィルタ（ローパス）
def lowpass(x, samplerate, fp, fs, gpass, gstop):
    fn = samplerate / 2                           #ナイキスト周波数
    wp = fp / fn                                  #ナイキスト周波数で通過域端周波数を正規化
    ws = fs / fn                                  #ナイキスト周波数で阻止域端周波数を正規化
    N, Wn = signal.buttord(wp, ws, gpass, gstop)  #オーダーとバターワースの正規化周波数を計算
    b, a = signal.butter(N, Wn, "low")            #フィルタ伝達関数の分子と分母を計算
    y = signal.filtfilt(b, a, x)                  #信号に対してフィルタをかける
    return y                                      #フィルタ後の信号を返す

In [ ]:
species=train.species.value_counts()
birds=train.ebird_code.unique()
path="../input/birdsong-recognition/train_audio/"
path_ex_test = "../input/birdsong-recognition/example_test_audio/"

In [ ]:
top_rate = 0.2 # e.g., 鳥Aのサンプル数100 -> top_rate=0.2で20サンプル取り出す
min_sample_num = 10 # 最小サンプル数 top_rate x 鳥Aの全サンプル数 < min_sample_num の場合、鳥Aのサンプルを全て使う。少ないサンプルはなるべく使いたいため
high_pass = True
high_pass_cfg = {
    'fp': 2000,       #通過域端周波数[Hz]
    'fs': 1500,       #阻止域端周波数[Hz]
    'gpass': 3,       #通過域端最大損失[dB]
    'gstop': 40      #阻止域端最小損失[dB]
}
low_pass = False
low_pass_cfg = {
    'fp': 2000,       #通過域端周波数[Hz]
    'fs': 2500,       #阻止域端周波数[Hz]
    'gpass': 3,       #通過域端最大損失[dB]
    'gstop': 40      #阻止域端最小損失[dB]
}

sec = 5
get_end = False # これがtrueなら音声の末尾sec秒を取得
sort_items = {
    'date': "down",
    'bird_seen': "down",
    'rating': "down"
}

sampling_rate = 32000

In [ ]:
for bird_name in tqdm(birds):
    print(bird_name)
    df = train.query("ebird_code == @bird_name")
    top_df = get_top_data_info(df, sort_items, top_rate, min_sample_num = 10)
    files = top_df['filename'].values
    for file in files:
        audio_path = os.path.join(path, bird_name, file)
        x , sr = librosa.load(audio_path, sr = sampling_rate)
        print(sr)
        if high_pass:
            x = highpass(x, sr, high_pass_cfg['fp'], high_pass_cfg['fs'], high_pass_cfg['gpass'], high_pass_cfg['gstop'])
        elif low_pass:
            x = lowpass(x, sr, low_pass_cfg['fp'], low_pass_cfg['fs'], low_pass_cfg['gpass'], low_pass_cfg['gstop'])
        audio_sec = len(x) / sr
        x = np.asfortranarray(x) # なんかこれ入れないとエラー出る
        if audio_sec < sec:
            add_x = x
            while (len(x) / sr) < sec:
                x = np.r_[x, add_x]
            audio_sec = len(x) / sr
        # print("Audio Length: {}".format(audio_sec))
        if get_end == False:
            sub_x = x[:sec*sr]
        else:
            sub_x = x[-(sec*sr):]
        sub_x = librosa.stft(sub_x)
        Xdb = librosa.amplitude_to_db(abs(sub_x))
        if get_end == False:
            np.savetxt('{}_{}_First{}sec.csv'.format(bird_name, file.split('.')[0], sec), Xdb)
        else:
            np.savetxt('{}_{}_Last{}sec.csv'.format(bird_name, file.split('.')[0], sec), Xdb)